In [ ]:
from PIL import Image
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import notebook
import torchvision
from torchvision import transforms
from torchvision import models
from collections import Counter
from pathlib import Path
from sklearn.metrics import confusion_matrix, recall_score
import seaborn as sn
import pyarrow.parquet as pq
from skimage.filters import threshold_otsu
from skimage.transform import AffineTransform, SimilarityTransform, warp, resize

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
df_test = pd.read_csv('/kaggle/input/bengaliai-cv19/test.csv')

In [ ]:
def make_tensordataset_from_dfs(parquet_locs, label_loc=None):
    ids = []
    X = []
    for parquet_loc in parquet_locs:
        df = pd.read_parquet(parquet_loc)
        ids.extend(df.image_id.tolist())
        x = df.iloc[:, 1:].to_numpy(dtype=np.uint8)
        del df
        X.append(x)
    X = np.vstack(X)
    X = X.reshape(-1, 1, 137, 236)
    X = torch.from_numpy(X)
    ids = dict((s,i) for (i,s) in enumerate(ids))
    if label_loc is None:
        return TensorDataset(X)
    else:
        graphemes = torch.zeros(X.shape[0], dtype=torch.long)
        vowel_diacs = torch.zeros(X.shape[0], dtype=torch.long)
        consonant_diacs = torch.zeros(X.shape[0], dtype=torch.long)
        lbl_df = pd.read_csv(label_loc)
        for row in lbl_df.itertuples():
            if row.image_id not in ids:
                continue
            idx = ids[row.image_id]
            graphemes[idx] = row.grapheme_root
            vowel_diacs[idx] = row.vowel_diacritic
            consonant_diacs[idx] = row.consonant_diacritic
        return TensorDataset(X, graphemes, vowel_diacs, consonant_diacs)

In [ ]:
class DatasetWithImageTransforms(Dataset):
    def __init__(self, ds, transforms):
        super(DatasetWithImageTransforms, self).__init__()
        self.ds = ds
        self.tr = transforms
        self.nt = len(self.ds[0])
        self.ln = len(self.ds)
        
    def __getitem__(self, index):
        img, *rest = self.ds[index]
        img = self.tr(img)
        return (img,) + tuple(rest)
    
    def __len__(self):
        return self.ln

In [ ]:
# Returns binary image
def thresh(img):
    thresh_val = int(threshold_otsu(img))
    img = (img > thresh_val)
    return img

# For binary image
def bounding_box(img):
    img = thresh(img).astype(np.uint8)
    # find the min value of each column
    col_min_val = np.min(img, axis=0)
    # find the min value of each row
    row_min_val = np.min(img, axis=1)
    # argwhere finds the non-zero elements we want to find the zero elements (zeros are part of character)
    col = np.argwhere(1 - col_min_val).flatten()
    row = np.argwhere(1 - row_min_val).flatten()
    return row.min(), row.max(), col.min(), col.max()

def scale_to_bb(img):
    height = img.shape[0]
    width = img.shape[1]
    t, b, l, r = bounding_box(img)
    box_width = r - l
    box_height = b - t
    t, l = max(0, t - 10), max(0, l - 10)
    b, r = min(height, b + 10), min(width, r + 10)
#     print(l, r, t, b)
    img = resize(img[t:b, l:r], output_shape=(256, 256), preserve_range=True, order=3, cval=1.0)
    return img

def random_scale(img):
    height = img.shape[0]
    width = img.shape[1]
    t, b, l, r = bounding_box(img)
    box_width = r - l
    box_height = b - t
    max_width_scale = (box_width + min(l, width - r)) / box_width
    max_height_scale = (box_height + min(t, height - b)) / box_height
    max_scale = min(max_width_scale, max_height_scale)
    min_scale = min(1.0, 0.25 * max((height / box_height), (width / box_width)))
    scale = random.uniform(min_scale, max_scale)
    tfm = SimilarityTransform(
        scale=(scale, scale),
    )
    img = warp(img, tfm.inverse, cval=1.0, order=3)
    return img

def random_translate(img):
    height = img.shape[0]
    width = img.shape[1]
    t, b, l, r = bounding_box(img)
    box_width = r - l
    box_height = b - t
    translate_height = random.uniform(-t, height - b)
    translate_width = random.uniform(-l, width - r)
    tfm = SimilarityTransform(
        translation=(translate_width, translate_height),
    )
    img = warp(img, tfm.inverse, cval=1.0, order=3)
    return img

def random_rotate_and_shear(img):
    max_theta = math.pi / 16
    theta = random.uniform(-max_theta, max_theta)
    max_shear_theta = math.pi / 8
    shear_theta = random.uniform(-max_shear_theta, max_shear_theta)
    tfm = AffineTransform(rotation=theta, shear=shear_theta)
    img = warp(img, tfm.inverse, cval=1.0, order=3)
    return img

def invert_color(t):
    t.mul_(-1)
    t.add_(255)
    return t

def affine_transforms(img):
    img = img.reshape(137, 236).numpy()
    img = thresh(img).astype(np.float32)
#     img = random_translate(random_scale(random_rotate_and_shear(img)))
    img = random_translate(random_scale(img))
    img = img.reshape(1, 137, 236)
    return torch.from_numpy(img)

def tfms(img):
    img = img.reshape(137, 236).numpy()
    img = thresh(img).astype(np.float32)
    img = scale_to_bb(img)
    img = img.reshape(1, 256, 256)
    return torch.from_numpy(img)

def va_tfms(img):
    img = img.reshape(137, 236).numpy()
    img = thresh(img).astype(np.float32)
    img = img.reshape(1, 137, 236)
    return torch.from_numpy(img)

affine_transforms = transforms.Lambda(affine_transforms)
mult = transforms.Lambda(lambda img: img * 255)
to_float = transforms.Lambda(lambda img: img.float())
invert_color = transforms.Lambda(invert_color)
tfms = transforms.Compose([
#     transforms.RandomApply([
        tfms,
#         mult,
#     ], p=0.90),
    to_float,
])

In [ ]:
# ds = make_tensordataset_from_dfs(['/kaggle/input/bengaliai-cv19/test_image_data_{}.parquet'.format(i) for i in range(4)])

In [ ]:
n_graphemes, n_vowel_diacs, n_consonant_diacs = 168, 11, 7

In [ ]:
def make_linear_block_old(in_size, out_size):
    block = nn.Sequential(
        nn.Linear(in_size, out_size), 
        nn.ReLU(), 
        nn.BatchNorm1d(num_features=out_size),
    )
    nn.init.xavier_normal_(block[0].weight.data)
    nn.init.zeros_(block[0].bias.data)
    return block

def make_ff_predictor_old(in_size, intermediate_size, out_size, layer_count):
    layers = [make_linear_block_old(in_size, intermediate_size)]
    for i in range(layer_count):
        layers.append(make_linear_block_old(intermediate_size, intermediate_size))
    layers.append(make_linear_block_old(intermediate_size, out_size))
    layers = nn.Sequential(*layers)
    return layers

def make_squeeze_predictor(in_size, out_size):
    return nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Conv2d(in_size, out_size, kernel_size=(1, 1)),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(1,1)),
            nn.Flatten()
        )

class BanglaHandwrittenGraphemeNN(nn.Module):
    def __init__(self):
        super(BanglaHandwrittenGraphemeNN, self).__init__()
#         base = models.squeezenet1_0(pretrained=True).features
        base = models.resnet18(pretrained=False)
#         base.load_state_dict(torch.load('/kaggle/input/pretrained-pytorch-models/resnet18-5c106cde.pth'))
        base.fc = nn.Identity()
        self.base = base
        feature_size = 512
        self.grapheme_predictor = make_ff_predictor_old(feature_size, 512, n_graphemes, 2)
        self.vowel_diac_predictor = make_ff_predictor_old(feature_size, 512, n_vowel_diacs, 1)
        self.consonant_diacs = make_ff_predictor_old(feature_size, 512, n_consonant_diacs, 1)

    def convert_to_grayscale(self):
        with torch.no_grad():
            conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            conv1.weight.data = torch.sum(model.base.conv1.weight.data, dim=1, keepdim=True)
            self.base.conv1 = conv1
            
    def freeze(self):
        for p in self.base.parameters():
            p.requires_grad = False
        
    def unfreeze(self):
        for p in self.base.parameters():
            p.requires_grad = True
        
    def forward(self, x):
        with torch.no_grad():
            features = self.base(x)
        g_pred = self.grapheme_predictor(features)
        v_pred = self.vowel_diac_predictor(features)
        c_pred = self.consonant_diacs(features)
        return g_pred, v_pred, c_pred

In [ ]:
def make_linear_block(in_size, out_size, dropout, bn):
    if dropout:
        if bn:
            block = nn.Sequential(
                nn.Linear(in_size, out_size), 
                nn.Dropout(0.5),
                nn.ReLU(), 
                nn.BatchNorm1d(num_features=out_size),
            )
        else:
            block = nn.Sequential(
                nn.Linear(in_size, out_size), 
                nn.Dropout(0.5),
                nn.ReLU(), 
            )
    else:
        if bn:
            block = nn.Sequential(
                nn.Linear(in_size, out_size),
                nn.ReLU(), 
                nn.BatchNorm1d(num_features=out_size),
            )
        else:
            block = nn.Sequential(
                nn.Linear(in_size, out_size),
                nn.ReLU(),
            )
    nn.init.xavier_normal_(block[0].weight.data)
    nn.init.zeros_(block[0].bias.data)
    return block

def make_ff_predictor(in_size, intermediate_size, out_size, layer_count, dropout, bn):
    layers = [make_linear_block(in_size, intermediate_size, dropout, bn)]
    for i in range(layer_count):
        layers.append(make_linear_block(intermediate_size, intermediate_size, dropout, bn))
    layers.append(nn.Linear(intermediate_size, out_size))
    layers = nn.Sequential(*layers)
    return layers

def make_squeeze_predictor(in_size, out_size):
    return nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Conv2d(in_size, out_size, kernel_size=(1, 1)),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(1,1)),
            nn.Flatten(), 
        )

class MultiTaskNNR18(nn.Module):
    def __init__(self, n_classes_tasks, depth_tasks, dropout=False, bn=False):
        super(MultiTaskNNR18, self).__init__()
#         base = models.squeezenet1_0(pretrained=True).features
        base, feature_size = models.resnet18(pretrained=False), 512
#         base = models.wide_resnet101_2(pretrained=True)
        base.fc = nn.Identity()
#         base = models.densenet121(pretrained=True)
#         base.classifier = nn.Identity()
        self.base = base
        self.task_predictors = nn.ModuleList([
            make_ff_predictor(feature_size, 512, n_classes, depth, dropout, bn) 
            for n_classes, depth in zip(n_classes_tasks, depth_tasks)
        ])
        self.n_classes_tasks = n_classes_tasks
        self.depth_tasks = depth_tasks

    def convert_to_grayscale(self):
        with torch.no_grad():
            conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            conv1.weight.data = torch.sum(self.base.conv1.weight.data, dim=1, keepdim=True)
            self.base.conv1 = conv1
#             conv1.weight.data = torch.sum(self.base.features.conv0.weight.data, dim=1, keepdim=True)
#             self.base.features.conv0 = conv1
            
    def freeze(self):
        for p in self.base.parameters():
            p.requires_grad = False
        
    def unfreeze(self):
        for p in self.base.parameters():
            p.requires_grad = True
        
    def forward(self, x):
        features = self.base(x)
        preds = [predictor(features) for predictor in self.task_predictors]
        return preds

In [ ]:
class MultiTaskNN(nn.Module):
    def __init__(self, base, task_predictors):
        super(MultiTaskNN, self).__init__()
        self.base = base
        self.task_predictors = nn.ModuleList(task_predictors)
        
    def freeze(self):
        for p in self.base.parameters():
            p.requires_grad = False
        
    def unfreeze(self):
        for p in self.base.parameters():
            p.requires_grad = True
        
    def forward(self, x):
        features = self.base(x)
        preds = [predictor(features) for predictor in self.task_predictors]
        return preds

In [ ]:
base = torch.jit.load('/kaggle/input/bhgd-r152-trace/resnet152-corrected-more-transforms-base.trace')
num_tasks = 3
task_predictors = [torch.jit.load('/kaggle/input/bhgd-r152-trace/resnet152-corrected-more-transforms-task-predictor-{}.trace'.format(i)) for i in range(num_tasks)]
model1 = MultiTaskNN(base, task_predictors)

In [ ]:
base, feature_size = torchvision.models.resnet152(pretrained=False), 2048
base.fc = nn.Identity()
conv1 = nn.Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
conv1.weight.data = torch.sum(base.conv1.weight.data, dim=1, keepdim=True)
base.conv1 = conv1

n_classes_tasks = [n_graphemes, n_vowel_diacs, n_consonant_diacs]

task_predictors = [nn.Linear(feature_size, n_classes, bias=False) for n_classes in n_classes_tasks]

model2 = MultiTaskNN(base, task_predictors).to(device)

model2.load_state_dict(torch.load('/kaggle/input/bhgd-r152-itstrat-mixup-partial/resnet152-itstrat-mixup.pth', map_location=device))

In [ ]:
base, feature_size = torchvision.models.resnet152(pretrained=False), 2048
base.fc = nn.Identity()
conv1 = nn.Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
conv1.weight.data = torch.sum(base.conv1.weight.data, dim=1, keepdim=True)
base.conv1 = conv1

task_predictors = [nn.Linear(feature_size, n_classes, bias=False) for n_classes in n_classes_tasks]

model3 = MultiTaskNN(base, task_predictors).to(device)

model3.load_state_dict(torch.load('/kaggle/input/bhgd-r152-itstrat-mixup-partial/resnet152-itstrat-mixup-120-1e-3-0.2-16-8.pth', map_location=device))

In [ ]:
class MultiTaskEnsemble(nn.Module):
    def __init__(self, models):
        super(MultiTaskEnsemble, self).__init__()
        self.models = nn.ModuleList(models)
        self.n = len(models)

    def forward(self, x):
        preds = [m(x) for m in self.models]
        preds = [[pred[i] for pred in preds] for i in range(len(preds[0]))]
        preds = [torch.stack(pred).mean(dim=0) for pred in preds]
        return preds
#         xes = [sum(x[i] for x in xes) / self.n for i in range(len(xes[0]))]
#         return xes

In [ ]:
model = BanglaHandwrittenGraphemeNN().to(device)
model.convert_to_grayscale()
model.load_state_dict(torch.load('/kaggle/input/bhgd-r18-best/model.pth', map_location=device))
model_r18_best = model

In [ ]:
model = MultiTaskNNR18([n_graphemes, n_vowel_diacs, n_consonant_diacs], [2, 1, 1], bn=True).to(device)
model.convert_to_grayscale()
model.load_state_dict(torch.load('/kaggle/input/bhgd-r18-best-clones/resnet18-original-lossweight-111.pth', map_location=device))
model_r18_111 = model

In [ ]:
model = MultiTaskNNR18([n_graphemes, n_vowel_diacs, n_consonant_diacs], [2, 1, 1], bn=True).to(device)
model.convert_to_grayscale()
model.load_state_dict(torch.load('/kaggle/input/bhgd-r18-best-clones/resnet18-original-lossweight-211.pth', map_location=device))
model_r18_211 = model

In [ ]:
model = MultiTaskNNR18([n_graphemes, n_vowel_diacs, n_consonant_diacs], [2, 1, 1], bn=True).to(device)
model.convert_to_grayscale()
model.load_state_dict(torch.load('/kaggle/input/bhgd-r18-best-clones/resnet18-original-80ep.pth', map_location=device))
model_r18_80ep = model

In [ ]:
ens1 = MultiTaskEnsemble([model_r18_best, model_r18_111, model_r18_211, model_r18_80ep]).to(device)

In [ ]:
ens2 = MultiTaskEnsemble([model1, model2, model3]).to(device)

In [ ]:
def load_model(name, num_tasks):
    base = torch.jit.load(name + '-base.trace')
    task_predictors = [torch.jit.load(name + '-task-predictor-{}.trace'.format(i)) for i in range(num_tasks)]
    return MultiTaskNN(base, task_predictors)

In [ ]:
model1 = load_model('/kaggle/input/bhgd-final-attempt/seresnext101-mixup-convtail-small', 3)

In [ ]:
model2 = load_model('/kaggle/input/bhgd-final-attempt/seresnext101-mixup-convtail', 3)

In [ ]:
model3 = load_model('/kaggle/input/bhgd-final-attempt/seresnext101-mixup-40ep-rotate-shear', 3)

In [ ]:
model4 = load_model('/kaggle/input/bhgd-final-attempt/seresnext101-mixup-convtail-small-40ep', 3)

In [ ]:
ens3 = MultiTaskEnsemble([model1, model2, model3, model4]).to(device)

In [ ]:
models = [ens1, ens2, ens3]

In [ ]:
model = MultiTaskEnsemble(models).to(device)

In [ ]:
model.eval();

In [ ]:
results = []

In [ ]:
with torch.no_grad():
    for i in range(4):
        te_ds = make_tensordataset_from_dfs(
            ['/kaggle/input/bengaliai-cv19' + '/test_image_data_{}.parquet'.format(i)]
        )
        # TTA
        te_ds = DatasetWithImageTransforms(te_ds, tfms)
        num_tta = 1
        te_dl = DataLoader(te_ds, batch_size=256, num_workers=2, pin_memory=True)
        tta_results = [[] for i in range(num_tta)]
        for i in range(num_tta):
            for imgs, in te_dl:
                imgs = imgs.to(device)
#                 imgs = imgs / 255.0
                g_pred, v_pred, c_pred, *rest = model(imgs)
                tta_results[i].append([g_pred, v_pred, c_pred])
        
        for i in range(len(te_dl)):
            g_preds = [results[i][0] for results in tta_results]
            v_preds = [results[i][1] for results in tta_results]
            c_preds = [results[i][2] for results in tta_results]
            g_pred = torch.stack(g_preds).mean(dim=0)
            v_pred = torch.stack(v_preds).mean(dim=0)
            c_pred = torch.stack(c_preds).mean(dim=0)
            g_pred = g_pred.argmax(1).tolist()
            v_pred = v_pred.argmax(1).tolist()
            c_pred = c_pred.argmax(1).tolist()
            
            for g, v, c in zip(g_pred, v_pred, c_pred):
                results.extend([c, g, v])
        del imgs
        del te_ds
        del te_dl
        del tta_results

In [ ]:
del models

In [ ]:
del model

In [ ]:
submission_df = pd.read_csv('/kaggle/input/bengaliai-cv19' + '/sample_submission.csv')

In [ ]:
submission_df['target'] = results

In [ ]:
submission_df.to_csv('./submission.csv', index=False)

In [ ]:
submission_df